**NN**

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define training function
def train_model(model, train_loader, val_loader,optimizer, epochs=10):
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        for batch in train_loader:
            images, labels = batch[0].to(device), batch[1].to(device)  # Move batch to device
            # make the gradients as zero before you start with this batch
            optimizer.zero_grad()
            outputs = model(images)
            # feed the outputs predicted by the model to the criterion to calculate the loss function and then backpropagate
            loss = criterion(outputs, labels)
            loss.backward()
            # update the weights accordingly
            optimizer.step()

        val_loss, val_acc = evaluate(model, val_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# Define evaluation function
def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)  # Move batch to device
            outputs = model(images)
            loss = F.cross_entropy(outputs, labels)
            total_loss += loss.item()
            # basically one out of 10 with the highest probability would correspond to output of that
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    average_loss = total_loss / len(val_loader)
    return average_loss, accuracy


In [19]:
class CIFAR10MLP(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2,hidden_size3, output_size):
        super(CIFAR10MLP, self).__init__()
        # define the first and second fully connected layer and the output layer as well
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2,hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        # Apply the relu activation function for first and second layer only
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x= F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [20]:

# Load CIFAR-10 dataset
dataset = CIFAR10(root='data/', download=True, transform=ToTensor())
test_dataset = CIFAR10(root='data/', train=False, transform=ToTensor())
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)





Files already downloaded and verified


In [21]:
# Model parameters
input_size = 3 * 32 * 32  # 3 channels, 32x32 image size
hidden_size1 = 512
hidden_size2 = 256
hidden_size3 =128
output_size = 10

# # Initialize and train the model
model = CIFAR10MLP(input_size, hidden_size1, hidden_size2,hidden_size3, output_size)
model = model.to(device)
lr=0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
train_model(model, train_loader, val_loader,optimizer,epochs=30)
loss,test_accuracy = evaluate(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.4f}")

Epoch [1/30], Val Loss: 1.7827, Val Acc: 0.3610
Epoch [2/30], Val Loss: 1.6489, Val Acc: 0.4136
Epoch [3/30], Val Loss: 1.6399, Val Acc: 0.4228
Epoch [4/30], Val Loss: 1.5613, Val Acc: 0.4458
Epoch [5/30], Val Loss: 1.5252, Val Acc: 0.4584
Epoch [6/30], Val Loss: 1.5089, Val Acc: 0.4598
Epoch [7/30], Val Loss: 1.5289, Val Acc: 0.4510
Epoch [8/30], Val Loss: 1.5132, Val Acc: 0.4666
Epoch [9/30], Val Loss: 1.4510, Val Acc: 0.4860
Epoch [10/30], Val Loss: 1.4650, Val Acc: 0.4836
Epoch [11/30], Val Loss: 1.4154, Val Acc: 0.5050
Epoch [12/30], Val Loss: 1.4190, Val Acc: 0.5014
Epoch [13/30], Val Loss: 1.4010, Val Acc: 0.5118
Epoch [14/30], Val Loss: 1.3880, Val Acc: 0.5120
Epoch [15/30], Val Loss: 1.4050, Val Acc: 0.5134
Epoch [16/30], Val Loss: 1.4136, Val Acc: 0.5050
Epoch [17/30], Val Loss: 1.4019, Val Acc: 0.5202
Epoch [18/30], Val Loss: 1.4069, Val Acc: 0.5106
Epoch [19/30], Val Loss: 1.4101, Val Acc: 0.5202
Epoch [20/30], Val Loss: 1.4135, Val Acc: 0.5248
Epoch [21/30], Val Loss: 1.39

**CNN**

In [23]:
import torch.nn as nn

class CIFAR10CNN(nn.Module):
    def __init__(self):
        super(CIFAR10CNN, self).__init__()
        # convolutional layer (sees 32x32x3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        # convolutional layer (sees 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        # convolutional layer (sees 8x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        # linear layer (64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(-1, 64 * 4 * 4)
        # add dropout layer
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x


# # Initialize and train the model
model1 = CIFAR10CNN()
model1 = model1.to(device)
lr=0.001
optimizer = torch.optim.Adam(model1.parameters(), lr=lr)
train_model(model1, train_loader, val_loader,optimizer,epochs=30)


Epoch [1/30], Val Loss: 1.4198, Val Acc: 0.4896
Epoch [2/30], Val Loss: 1.2678, Val Acc: 0.5484
Epoch [3/30], Val Loss: 1.1408, Val Acc: 0.5984
Epoch [4/30], Val Loss: 1.0714, Val Acc: 0.6300
Epoch [5/30], Val Loss: 1.0181, Val Acc: 0.6570
Epoch [6/30], Val Loss: 0.9613, Val Acc: 0.6760
Epoch [7/30], Val Loss: 0.9496, Val Acc: 0.6676
Epoch [8/30], Val Loss: 0.9267, Val Acc: 0.6854
Epoch [9/30], Val Loss: 0.9175, Val Acc: 0.6956
Epoch [10/30], Val Loss: 0.9013, Val Acc: 0.7026
Epoch [11/30], Val Loss: 0.9070, Val Acc: 0.7030
Epoch [12/30], Val Loss: 0.8875, Val Acc: 0.7152
Epoch [13/30], Val Loss: 0.9644, Val Acc: 0.6992
Epoch [14/30], Val Loss: 0.8992, Val Acc: 0.7200
Epoch [15/30], Val Loss: 0.9586, Val Acc: 0.7130
Epoch [16/30], Val Loss: 0.9999, Val Acc: 0.7166
Epoch [17/30], Val Loss: 1.0997, Val Acc: 0.7136
Epoch [18/30], Val Loss: 1.1292, Val Acc: 0.7152
Epoch [19/30], Val Loss: 1.2220, Val Acc: 0.7210
Epoch [20/30], Val Loss: 1.3143, Val Acc: 0.7146
Epoch [21/30], Val Loss: 1.40

In [24]:
loss1,test_accuracy1 = evaluate(model1, test_loader)
print(f"Test Accuracy: {test_accuracy1:.4f}")


Test Accuracy: 0.7228


**VGG16**

In [25]:
from torchvision import models
import torch.optim as optim

# Load the pre-trained VGG-16 model
model2 = models.vgg16(pretrained=True)
model2= model2.to(device)


# Freeze all the layers in the pre-trained model
for param in model2.parameters():
    param.requires_grad = False

# Unfreeze the last few layers
for param in model2.features[-4:].parameters():
    param.requires_grad = True

# Modify the last layer to match the number of classes in the CIFAR-10 dataset
num_features = model2.classifier[6].in_features
features = list(model2.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, 10)]) # Add our layer with 10 outputs
model2.classifier = nn.Sequential(*features) # Replace the model classifier

# Define the loss function and the optimizer
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model2.parameters()), lr=0.001, momentum=0.9)
model2 = model2.to(device)
train_model(model2, train_loader, val_loader,optimizer,epochs=30)

loss2,test_accuracy2 = evaluate(model2, test_loader)
print(f"Test Accuracy: {test_accuracy2:.4f}")





/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/30], Val Loss: 1.1002, Val Acc: 0.6150
Epoch [2/30], Val Loss: 1.0139, Val Acc: 0.6518
Epoch [3/30], Val Loss: 0.9501, Val Acc: 0.6650
Epoch [4/30], Val Loss: 0.9117, Val Acc: 0.6822
Epoch [5/30], Val Loss: 0.8914, Val Acc: 0.6906
Epoch [6/30], Val Loss: 0.8760, Val Acc: 0.6964
Epoch [7/30], Val Loss: 0.8586, Val Acc: 0.7018
Epoch [8/30], Val Loss: 0.8559, Val Acc: 0.7022
Epoch [9/30], Val Loss: 0.8480, Val Acc: 0.7024
Epoch [10/30], Val Loss: 0.8390, Val Acc: 0.7108
Epoch [11/30], Val Loss: 0.8279, Val Acc: 0.7122
Epoch [12/30], Val Loss: 0.8215, Val Acc: 0.7160
Epoch [13/30], Val Loss: 0.8114, Val Acc: 0.7188
Epoch [14/30], Val Loss: 0.8337, Val Acc: 0.7106
Epoch [15/30], Val Loss: 0.8306, Val Acc: 0.7110
Epoch [16/30], Val Loss: 0.8205, Val Acc: 0.7150
Epoch [17/30], Val Loss: 0.8064, Val Acc: 0.7274
Epoch [18/30], Val Loss: 0.8048, Val Acc: 0.7252
Epoch [19/30], Val Loss: 0.8330, Val Acc: 0.7180
Epoch [20/30], Val Loss: 0.8198, Val Acc: 0.7170
Epoch [21/30], Val Loss: 0.82

After conducting experiments, it was found that the Multi-Layer Perceptron (MLP) model achieved a final accuracy of 50 percent on the dataset. However, it was observed that the MLP model has a large number of parameters. This becomes problematic, especially when dealing with images of larger sizes, as the model may overfit due to the sheer volume of parameters involved. Overfitting occurs when a model learns to memorize the training data rather than generalize well to unseen data.

In contrast, the Convolutional Neural Network (CNN) model yielded a higher accuracy of 70 percent. There is potential for further improvement by increasing the number of training epochs. However, for the sake of comparison, the training was stopped at 70 percent accuracy. CNNs are particularly well-suited for image classification tasks due to their ability to automatically learn hierarchical patterns and features from the input data.

Additionally, another approach involved utilizing a pretrained VGG16 model. It was found that this approach resulted in higher accuracy compared to both the MLP and CNN models when trained from scratch. Transfer learning, as demonstrated by using a pretrained model like VGG16, allows for leveraging knowledge learned from one task or dataset to improve performance on another task or dataset. This highlights the effectiveness of leveraging pretrained models, especially in scenarios where limited labeled data or computational resources are available.